# CNN Annotation with Napari

### Welcome! 

This notebook allows you to import your raw microscopy data into [_Napari_](https://napari.org/ "Napari: a fast, interactive, multi-dimensional image viewer for Python"), a fast, interactive, multi-dimensional image viewer for Python. Follow the step-wise instructions to annotate your large, multi-dimensional images and extract single-cell image patches corresponding to different states of the cell cycle. 


In [1]:
import io
import os
import json
import napari
import numpy as np

from enum import Enum
from magicgui import magicgui
from skimage.io import imread
from pathlib import Path

from napari.layers import Image
from datetime import datetime

from zipfile import ZipFile
from skimage.external.tifffile import TiffWriter


### Provide the directory for the movie you wish to annotate:

In this example, we provide an example 50-frame long crop (600 x 450 pixels) of a time-lapse microscopy movie ```MDCK_H2B_GFP_movie.tif``` of MDCK cells expressing an *H2B-GFP* fluorescent tag, which visualises the nuclei of the individual cells. This movie has dimensions (*i.e.* shape) of ```(50, 450, 600)```. When loading your own movie, make sure it is saved as a ```tif``` file and provide its absolute path.

Unit conversion: *1 µm = 3 pixels.*


In [2]:
filename = "../MDCK_H2B_GFP_movie.tif"
print (filename)


../MDCK_H2B_GFP_movie.tif


### Load the image data from the movie:

In [3]:
data = imread(filename)
metadata = {"filename": filename}


### Define the cell cycle states which you'd like your CNN to categorise:

Create an object with enumerated classes which you want to classify. In this particular example, we provide 4 classes of actively dividing cells (*i.e. **interphase** pooled for G1-, S- and G2-phases & **pro(meta)phase, metaphase & ana(telo)phase** for specific phases of cell division*) and 1 class for ceasing cells (*i.e. **apoptosis***). 

To familiarise yourself with the typical cell morphologies & chromatin condensation of an actively dividing cell, please see the example images below:

![image](../cell_cycle_states.png)


In [4]:
class CellState(Enum):
    Interphase = 0
    Prometaphase = 1
    Metaphase = 2
    Anaphase = 3
    Apoptosis = 4


### Assign the colours to individual labelled states:

The default settings will follow the ```matplotlib``` standard colour library with {"blue", "orange", "green", "red", "purple"}. 

*Note:* When changing the default setting or defining new colour palettes, please specify the [HEX code](https://www.color-hex.com/ "HEX Color Codes") for new colours.


In [5]:
COLOR_CYCLE = [
    '#1f77b4', # blue
    '#ff7f0e', # orange
    '#2ca02c', # green
    '#d62728', # red
    '#9467bd', # purple
]

### This function crops an image patch around the labelled points:

Default setting will crop a 64 x 64 pixel square image patch with the labelled point at the centre of the patch, i.e. 32 pixels up, 32 pixels down, 32 pixels left & 32 pixels right from the labelled coordinates. Please only alter with care.


In [6]:
def get_image_patch(layers, coords, shape=64):
    """ Get an image patch from the image layer data. """
    
    frame, y_coo, x_coo = [int(coo) for coo in coords]
    pad = shape // 2
    
    # Read the image & crop patch around the point:
    image = layers[0].data[frame]
    patch = np.array([row[x_coo - pad : x_coo + pad] for row in image[y_coo - pad : y_coo + pad]])
    
    # Check if patches are of specified shape:
    if patch.shape != (shape, shape):
        raise ValueError(f"Image patch doesn't have correct shape: {patch.shape}")
    
    return patch


### The annotator function:

In [7]:
def annotator(viewer):
    
    SESSION_TIME = datetime.now().strftime("%m-%d-%Y--%H-%M-%S")
    SESSION_NAME = f"annotation_{SESSION_TIME}"
    
    # add an empty points layer, with the same dimensions as the image data
    points_layer = viewer.add_points(
        name="Annotation", 
        properties={'State': [s.name for s in CellState]}, 
        ndim=data.ndim
    )

    points_layer.mode = 'add'
    points_layer.face_color = 'State'
    points_layer.face_color_cycle = COLOR_CYCLE
    points_layer.face_color_mode = 'cycle'
    # points_layer.n_dimensional = True
    
    @magicgui(
        call_button="Export",
        layout="horizontal",
        filename={"label": "Export path:"},  # custom label
    )
    
    def cnn_annotation_widget(
        filename=Path.home(),  # path objects are provided a file picker
        shape=64,
        use_visible_layers=True,
        state=CellState.Interphase,
    ):
        """ Export the annotations: """
        
        export_data = {'shape': shape}
        
        # find the visible image layers and export the metadata
        image_layers = [layer for layer in viewer.layers if isinstance(layer, Image)]
        for layer in image_layers:
            if use_visible_layers and layer.visible:
                export_data[layer.name] = layer.metadata
        
        # record the coordinates of the annotations 
        for idx in range(points_layer.data.shape[1]):
            export_data[f'coords-{idx}'] = points_layer.data[:, idx].tolist()
        
        # record the state labels of the annotations 
        export_data['labels'] = points_layer.properties['State'].tolist()
        
        # extract the image patches here
        with ZipFile(f"{SESSION_NAME}.zip", 'w') as myzip:
            for idx, patch_coords in enumerate(points_layer.data):
                
                patch_label = points_layer.properties['State'][idx]

                # grab the image patch
                image_patch = get_image_patch(image_layers, patch_coords, shape=shape)
                image_patch_fn = f"{patch_label}/{patch_label}_{SESSION_TIME}_{idx}.tif"
                
                # open a stream to write to the zip file
                stream = io.BytesIO()
                with TiffWriter(stream) as tif:
                    tif.save(image_patch)
                    stream_data = stream.getvalue()
                myzip.writestr(image_patch_fn, stream_data)
        
            # write out the json log to the zip file also
            stream = json.dumps(export_data, indent=2)
            myzip.writestr(f"{SESSION_NAME}.json", stream)
        
        print (f"JSON file & image patches have been exported.\n'annotation_{SESSION_NAME}.zip'")
        
        return locals().values()
    
    def _change_points_properties(event):
        """ Update the current properties of the points layer to reflect the currently selected state. """
        points_layer.current_properties['State'] = np.array([cnn_annotation_widget.state.value.name])
    
    cnn_annotation_widget.state.changed.connect(_change_points_properties)
    
    # add the magicgui dock widget 
    viewer.window.add_dock_widget(cnn_annotation_widget)
    
    @viewer.bind_key('.')
    def next_label(event=None):
        """ Increment the label in the GUI """
        new_state = (cnn_annotation_widget.state.value.value + 1) % len(CellState)
        cnn_annotation_widget.state.value = CellState(new_state)
        
    @viewer.bind_key(',')
    def previous_label(event=None):
        """ Decrement the label in the GUI """
        new_state = (cnn_annotation_widget.state.value.value - 1) % len(CellState)
        cnn_annotation_widget.state.value = CellState(new_state)


# Running the cell below will open *napari* in a separate Python window. 

*Note:* ***Please allow a few seconds for the GUI to load.***

In [8]:
with napari.gui_qt():
    
    viewer = napari.Viewer()
    viewer.add_image(data, name='GFP', metadata=metadata)
   
    annotator(viewer)
    

### Export the annotations:

1. Click on the ```Export``` button at the bottom right corner of the *napari* window when done annotating.
2. Close the *napari* window & quit the python GUI completely.
3. Doing so will cause this notebook's kernel to die. That's perfectly fine.

#### Done!